# Count Islands

* Given a binary matrix (1=land, 0=water) 
* Return the number of islands (connected land in the 4 directions)


<span style="color:orange"><b>The point:</b></span>

* One one cell of land is found, identify the rest using any graph traversal algo
* Here we use DFS
* Must mark cells as visited (val from 1 to -1)



**Complexity :**

| Time        | Space        |
|-------------|--------------|
| O(m*n)      | O(m*n)       |

* O(m*n) in time because the matrix is visited at most twice (search for land, DFS)
* O(m*n) in space because the size of the recursive stack (can grow up to m*n) 









<!-- <span style="color:red"><b>TODO : </b></span> 
* Add comments in code -->


<!-- * <span style="color:lime"><b>Preferred solution?</b></span>      -->



## V1

**About Rust :**
* Changes are in `deep_copy_recursive`
    * Reduce the number of `node.borrow()`
    * Use `Vec::with_capacity` for the vector of `neighbors`
* <span style="color:lime"><b>Preferred solution?</b></span>     
* **YES** : tested on the [Rust Playground](https://play.rust-lang.org/)

In [ ]:
use std::cell::RefCell;
use std::rc::Rc;
use std::collections::HashMap;

// Type alias for readability
type GraphNodeRef = Rc<RefCell<GraphNode>>;

#[derive(Debug)]
struct GraphNode {
    val: usize,
    neighbors: Vec<GraphNodeRef>,
}

fn from_adjacency_list(adj_list: &[Vec<usize>]) -> Vec<GraphNodeRef> {
    let nodes: Vec<GraphNodeRef> = (0..adj_list.len()).map(|i| Rc::new(RefCell::new(GraphNode { val: i, neighbors: vec![] }))).collect();

    for (i, neighbors) in adj_list.iter().enumerate() {
        let mut node_mut = nodes[i].borrow_mut();
        node_mut.neighbors = neighbors.iter().map(|&j| Rc::clone(&nodes[j])).collect();
    }
    nodes
}

fn print_graph(node: &GraphNodeRef) {
    use std::collections::HashSet;
    let mut visited = HashSet::new();

    fn deep_copy_recursive_print(node: &GraphNodeRef, visited: &mut HashSet<usize>) {
        let val = node.borrow().val;
        if visited.contains(&val) {
            return;
        }
        visited.insert(val);
        let neighbors: Vec<_> = node.borrow().neighbors.iter().map(|n| n.borrow().val).collect();
        println!("Node {} has neighbors {:?}", val, neighbors);
        for neighbor in &node.borrow().neighbors {
            deep_copy_recursive_print(neighbor, visited);
        }
    }

    deep_copy_recursive_print(node, &mut visited);
}

// I’m not nesting deep_copy_recursive() inside deep_copy() to stay consistent with the book’s style and to improve code readability.
fn deep_copy_recursive(node: &GraphNodeRef, clone_map: &mut HashMap<usize, GraphNodeRef>) -> GraphNodeRef {
    //  If this node was already cloned, then return this previously cloned node
    let node_borrowed = node.borrow();
    if let Some(clone) = clone_map.get(&node_borrowed.val) {
        return Rc::clone(clone);
    }

    // Clone the current node
    let cloned_node = Rc::new(RefCell::new(GraphNode {
        val: node_borrowed.val,
        neighbors: Vec::with_capacity(node_borrowed.neighbors.len()), /
    }));

    //  Store the current clone to ensure it doesn't need to be created again in future DFS calls
    clone_map.insert(node_borrowed.val, Rc::clone(&cloned_node));

    // Iterate through the neighbors of the current node to connect their clones to the current cloned node
    for neighbor in &node_borrowed.neighbors {
        let cloned_neighbor = deep_copy_recursive(neighbor, clone_map);
        cloned_node.borrow_mut().neighbors.push(cloned_neighbor);
    }
    cloned_node
}

// Public deep copy function
fn deep_copy(start_node: &GraphNodeRef) -> GraphNodeRef {
    let mut clone_map = HashMap::new();
    deep_copy_recursive(start_node, &mut clone_map)
}


fn main() {
    let matrix = [
        vec![1, 1, 0, 0],
        vec![1, 1, 0, 0],
        vec![0, 0, 1, 1],
        vec![0, 0, 1, 1],
    ];

    println!("Number of islands = {}", count_islands(&matrix));
    // let my_graph = from_adjacency_list(&adjacency_list);
    // let my_start_node = &my_graph[0];

    // let my_cloned_start = deep_copy(my_start_node);
    
    // print_graph(&my_cloned_start);

}